### generate the pandas data

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from covid_screening_quantify import quantify_all_fovs, combine_all_quantify, write_conditions

** TORCH CUDA version installed and working. **


In [ ]:
# test parameter
#image_folder = '/home/haoxu/data/Images'

In [ ]:
#quantify_all_fovs(image_folder)

### combine csv data of all images

In [ ]:
#processed_folder = "/home/haoxu/data/Images_processed"

In [ ]:
#combine_all_quantify(processed_folder)

### generate the pandas data end

### get the conditions

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
# write the conditions to csv
processed_folder = "/home/haoxu/data/Images_processed"
quantify_csv_file = os.path.join(processed_folder, 'quantify_all.csv')
condition_xlsx_file = 'test/Plate map test plates 2020-11-25.xlsx'
write_conditions(quantify_csv_file, condition_xlsx_file)

### calculate z-prime statistic

In [24]:
import numpy as np
import pandas as pd
processed_folder = "/home/haoxu/data/Images_processed"
df = pd.read_csv(os.path.join(processed_folder, 'quantify_all.csv'))

In [27]:
# attribute if an column is DMSO infection treated or not
df  = df.assign(**dict.fromkeys(["DMSO_infection_treated"], np.nan))
DMSO_infection_treated_columns = ['23', '24']
df["DMSO_infection_treated"] = df.well_id.apply(lambda item: 0 if item[4:] in DMSO_infection_treated_columns else 1)
df

,well_id,well_fov,nuclei_file,er_file,virus_file,mask_file,cell_idx,cell_size,cell_integ,cell_mean,last4percentmean,Compound,DMSO_infection_treated
0,r12c13,r12c13f04,/home/haoxu/data/Images/r12c13f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r12c13f04_ma...,1,42,60980,1451.904762,2699,Fenbendazole,1
1,r12c13,r12c13f04,/home/haoxu/data/Images/r12c13f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r12c13f04_ma...,2,109,201791,1851.293578,4066,Fenbendazole,1
2,r12c13,r12c13f04,/home/haoxu/data/Images/r12c13f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r12c13f04_ma...,3,164,372553,2271.664634,4441,Fenbendazole,1
3,r12c13,r12c13f04,/home/haoxu/data/Images/r12c13f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r12c13f04_ma...,4,95,147001,1547.378947,3032,Fenbendazole,1
4,r12c13,r12c13f04,/home/haoxu/data/Images/r12c13f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r12c13f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r12c13f04_ma...,5,180,298425,1657.916667,2713,Fenbendazole,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1931559,r11c08,r11c08f04,/home/haoxu/data/Images/r11c08f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r11c08f04_ma...,1751,166,278463,1677.487952,3368,Ritonavir,1
1931560,r11c08,r11c08f04,/home/haoxu/data/Images/r11c08f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r11c08f04_ma...,1752,154,158748,1030.831169,1384,Ritonavir,1
1931561,r11c08,r11c08f04,/home/haoxu/data/Images/r11c08f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r11c08f04_ma...,1753,60,42584,709.733333,931,Ritonavir,1
1931562,r11c08,r11c08f04,/home/haoxu/data/Images/r11c08f04p01-ch1sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch3sk1fk1...,/home/haoxu/data/Images/r11c08f04p01-ch2sk1fk1...,/home/haoxu/data/Images_processed/r11c08f04_ma...,1754,33,17425,528.030303,601,Ritonavir,1


In [29]:
infection_rows = df[(df["DMSO_infection_treated"]==1)&(df['Compound']=='DMSO')]
infection_integration_values = infection_rows.cell_integ.to_list()
DMSO_infection_treated_values = {
    'mean': np.mean(infection_integration_values),
    'std': np.std(infection_integration_values)
}

noninfection_rows = df[(df["DMSO_infection_treated"]==0)&(df['Compound']=='DMSO')]
noninfection_integration_values = noninfection_rows.cell_integ.to_list()
DMSO_noninfection_treated_values = {
    'mean': np.mean(noninfection_integration_values),
    'std': np.std(noninfection_integration_values)
}

print('DMSO infection cells: ', DMSO_infection_treated_values)
print('DMSO noninfection cells: ', DMSO_noninfection_treated_values)

z_prime_factor = 1 - (
    3 * (DMSO_infection_treated_values['std'] + DMSO_noninfection_treated_values['std']) / \
    np.abs(DMSO_infection_treated_values['mean'] - DMSO_noninfection_treated_values['mean'])
)
print('z prime factor: ', z_prime_factor)

DMSO infection cells:  {'mean': 515395.17936383415, 'std': 402056.46709603845}
DMSO noninfection cells:  {'mean': 285682.80562161957, 'std': 223789.37982704124}
z prime factor:  -7.173427970738004


### define cell infection

In [ ]:
import pandas as pd
import os
from imageio import imread
from matplotlib import pyplot as plt
from skimage import segmentation, morphology, exposure
import numpy as np

In [ ]:
processed_folder = "/home/haoxu/data/Images_processed"
df = pd.read_csv(os.path.join(processed_folder, 'quantify_all.csv'))

In [ ]:
# use the max 4 percent in infection channel
plt.hist(df.last4percentmean.to_list(), range=(0, 20000), bins=800)

In [ ]:
# define the threshold value based on the plot above
threshold_value = 4000

In [ ]:
df  = df.assign(**dict.fromkeys(["Infected"], 0))
df.loc[df.last4percentmean > threshold_value, 'Infected'] = 1

### define cell infection end

### verify cell _infection

In [ ]:
well_fovs = list(set(df.well_fov.to_list()))
#well_fovs

In [ ]:
import random
from covid_screening_quantify import show_infections

In [ ]:
# show a random sample to see if the infected cells are correctly picked out.
# if not, fine tune the threshold_value and run
well_fov = 'r02c16f04'#random.choice(well_fovs)
print(well_fov)
img = show_infections(df, well_fov, dim_er=True, contour=True, show_er=False)
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(40, 40))
#ax = axes.ravel()
axes.imshow(img/img.max())

### get analysis result

In [ ]:
from covid_screening_quantify import plate_plots, plate_infection

In [ ]:
processed_folder = "/home/haoxu/data/Images_processed"
#infection_data = plate_infection(processed_folder, infection_threshold=3000)
#infection_data

### plot plate map for infections

In [ ]:
plate_plots(processed_folder, infection_threshold=4000)